In [1]:

import os
%pwd

'c:\\Users\\RETRETA\\Documents\\Python\\ML Upskill\\e2e-chest-cancer-classification\\research'

In [2]:
os.chdir("..")
%pwd

'c:\\Users\\RETRETA\\Documents\\Python\\ML Upskill\\e2e-chest-cancer-classification'

In [3]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/xret12/e2e-chest-cancer-classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "xret12"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "c05d9ad81ab52d7ad65f1197d91df0ab2f92d11f"

In [4]:
import tensorflow as tf

In [5]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        training = self.config.training
        evaluation = self.config.evaluation
        
        evaluation_config = EvaluationConfig(
            path_of_model=training.trained_model_path,
            training_data=training.training_data,
            all_params=self.params,
            mlflow_uri=evaluation.mlflow_uri,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

        return evaluation_config

In [9]:
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [10]:
class Evaluation:

    def __init__(self, config: EvaluationConfig):
        self.config = config


    def _valid_generator(self):
         
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluate(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()


    def save_score(self):
        scores = {
            "loss": self.score[0],
            "accuracy": self.score[1]
        }
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        print(tracking_url_type_store)

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                "loss": self.score[0],
                "accuracy": self.score[1]
            })
            
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [12]:
from cnnClassifier import logger

STAGE_NAME = "Stage: Model Evaluation"
try:
    logger.info(f">>>>>>>>>>>>>> {STAGE_NAME} STARTED <<<<<<<<<<<<<<<")
    config = ConfigurationManager()
    evaluation_config = config.get_evaluation_config()
    evaluation = Evaluation(config=evaluation_config)
    evaluation.evaluate()
    evaluation.log_into_mlflow()
    logger.info(f">>>>>>>>>>>>>> {STAGE_NAME} COMPLETED <<<<<<<<<<<<<<<")
    
except Exception as e:
    logger.exception(f"Exception raised while running {STAGE_NAME}: {e}")
    raise e

[2024-06-04 23:49:49,284: INFO: 3688520879: >>>>>>>>>>>>>> Stage: Model Evaluation STARTED <<<<<<<<<<<<<<<]
[2024-06-04 23:49:49,287: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-04 23:49:49,289: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-04 23:49:49,290: INFO: common: Path already exists: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 6s 811ms/step - loss: 10.2309 - accuracy: 0.5686
[2024-06-04 23:49:55,434: INFO: common: json file saved at: scores.json]
https


2024/06/04 23:49:56 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-06-04 23:49:57,496: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\RETRETA\AppData\Local\Temp\tmp_epe2twd\model\data\model\assets
[2024-06-04 23:49:57,945: INFO: builder_impl: Assets written to: C:\Users\RETRETA\AppData\Local\Temp\tmp_epe2twd\model\data\model\assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/06/04 23:50:57 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 5
Created version '5' of model 'VGG16Model'.


[2024-06-04 23:50:57,846: INFO: 3688520879: >>>>>>>>>>>>>> Stage: Model Evaluation COMPLETED <<<<<<<<<<<<<<<]
